# Параметрические критерии

### Вопрос 1
Существуют две версии одновыборочного Z-критерия для доли, отличающихся формулами для статистики. Одна из этих версий более точная, поэтому мы говорили только о ней, а вторую не обсуждали вообще. Какую из этих двух версий одновыборочных Z-критериев для доли мы проходили?

Версия множителей Лагранжа со статистикой 
$$Z = \frac {\hat{p}-p_{0}}{\sqrt{\frac{p_{0}(1-p_{0})}{n}}}$$

### Вопрос 2
Z-критерий для двух долей в связанных выборках использует только информацию о:

 * ~~количестве единиц в каждой из выборок и их объёме n~~
 * ~~количестве несовпадающих пар значений признаков — (0,1) и (1,0), и порядке их появления~~
 * количестве несовпадающих пар значений признаков — (0,1) и (1,0), и объёме выборок n
 * ~~количестве совпадающих пар значений признаков — (0,0) и (1,1), и объёме выборок n~~
 
### Вопрос 3
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [3]:
n1 = 34
s1 = 10

n2 = 16
s2 = 4

In [4]:
p1 = float(s1)/n1
p2 = float(s2)/n2

In [5]:
p = (p1*n1 + p2*n2)/(n1 + n2)

In [6]:
Z = (p1 - p2)/(p*(1-p)*(1./n1 + 1./n2))**0.5

In [16]:
import scipy.stats
import numpy as np
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [18]:
proportions_diff_z_test(Z, 'greater')

0.37293045872523534

### Вопрос 4
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными.
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

 * логистическая регрессия по признакам X1,X2,X3;
 * логистическая регрессия по признакам X4,X5,X6.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [30]:
import pandas as pd
from sklearn import cross_validation, linear_model
banknotes = pd.read_csv('banknotes.txt', sep = '\t')

In [26]:
train_banknotes, test_banknotes = cross_validation.train_test_split(banknotes, random_state = 1, test_size = 50)

In [31]:
clf1 = linear_model.LogisticRegression()
clf2 = linear_model.LogisticRegression()

In [33]:
clf1.fit(train_banknotes[['X1', 'X2', 'X3']], train_banknotes.real)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
clf2.fit(train_banknotes[['X4', 'X5', 'X6']], train_banknotes.real)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
test_banknotes['predictions1'] = clf1.predict(test_banknotes[['X1', 'X2', 'X3']])

In [38]:
test_banknotes['predictions2'] = clf2.predict(test_banknotes[['X4', 'X5', 'X6']])

In [44]:
test_banknotes['predictions1_accuracy'] = map(int, test_banknotes.predictions1 == test_banknotes.real)
test_banknotes['predictions2_accuracy'] = map(int, test_banknotes.predictions2 == test_banknotes.real)

In [46]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [47]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [49]:
proportions_diff_z_test(proportions_diff_z_stat_rel(test_banknotes.predictions1_accuracy, 
                                                    test_banknotes.predictions2_accuracy))

0.0032969384555543435

### Вопрос 5
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [50]:
proportions_diff_confint_rel(test_banknotes.predictions1_accuracy, 
                                                    test_banknotes.predictions2_accuracy)

(-0.30005479372038568, -0.059945206279614305)

### Вопрос 6
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.


In [56]:
mu0 = 525
sigma = 100

In [57]:
n = 100
mu = 541.4

In [59]:
import math
z = ((mu - mu0)/(sigma/math.sqrt(n)))
z

1.6399999999999977

In [60]:
p = 1 - scipy.stats.norm.cdf(z)

In [61]:
p

0.050502583474103968

### Вопрос 7
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [63]:
mu0 = 525
sigma = 100
n = 100
mu = 541.5
z = ((mu - mu0)/(sigma/math.sqrt(n)))
p = 1 - scipy.stats.norm.cdf(z)
p

0.049471468033648103